In [1]:
import pandas as pd

In [2]:
df_raw = pd.read_csv(r'M:\Data\OnBoard\Data and Reports\BART\As CSV\BART_Final_Database_Mar18_SUBMITTED_with_station_xy_with_first_board_last_alight NO POUND OR SINGLE QUOTE.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (4,13,17,26,32,33,36,38,39,40,51,53,55,59,72,78,79,82,83,85,86,87,102,109,115,126,128,130,138,140,146,159,160,161,162,163,164,165,166,167,168,169,170,171,172,177,207,208,212,214,215,216,217,218,219,220,221,222,223,224,230,231) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df_raw.copy()
df.head()

,ID,DATE_COMPLETED,TIME_COMPLETED,STANDARD_INTERV_INIT,STATION_SURVEYED_CODE,STATION_SURVEYED,Q1_START,OLD_ENOUGH4SURVEY,INTERCEPT_LOCATION_CODE,FIRST_ENTERED_BART_CODE,...,HOME_COUNTY_GIS,FARECATEGORYSUMMARY,FIRST_ENTER_BART_LAT,FIRST_ENTER_BART_LON,BART_EXIT_LAT,BART_EXIT_LON,FIRST_BOARD_LAT,FIRST_BOARD_LON,LAST_ALIGHT_LAT,LAST_ALIGHT_LON
0,160.0,2/17/2015,5:23:10 AM,JE,STA27,Montgomery St.,2/17/2015 5:16,YES,Intercepted at Exit,STA12,...,CONTRA COSTA,Disabled discount,37.970812,-122.030659,37.789394,-122.401088,37.970812,-122.030659,37.798591,-122.401074
1,161.0,2/17/2015,5:30:42 AM,MAW,STA2,16th St. Mission,2/17/2015 5:21,YES,TRANSFERRED_HERE,STA20,...,ALAMEDA,Regular,37.775128,-122.224225,37.638664,-122.417081,37.775128,-122.224225,37.638664,-122.417081
2,162.0,2/17/2015,5:28:46 AM,JJH,STA27,Montgomery St.,2/17/2015 5:21,YES,Intercepted at Entry Station,STA27,...,SAN FRANCISCO,Regular,37.789394,-122.401088,37.615910,-122.392721,37.789394,-122.401088,37.615910,-122.392721
3,163.0,2/17/2015,5:31:34 AM,NER,STA21,Glen Park,2/17/2015 5:23,YES,Intercepted at Entry Station,STA21,...,SAN FRANCISCO,Regular,37.733046,-122.433776,37.779532,-122.413788,37.733046,-122.433776,37.779532,-122.413788
4,164.0,2/17/2015,5:32:34 AM,JE,STA27,Montgomery St.,2/17/2015 5:24,YES,Intercepted at Entry Station,STA27,...,SAN FRANCISCO,Senior discount,37.789394,-122.401088,37.765023,-122.419743,37.778724,-122.414780,37.765023,-122.419743


In [4]:
for colname in ['RACE_OR_ETHNICITY [AMALA]', 'RACE_OR_ETHNICITY [ASIAN]',
                'RACE_OR_ETHNICITY [BLACK]', 'RACE_OR_ETHNICITY [HAWPI]',
                'RACE_OR_ETHNICITY [WHITE]', 'RACE_OR_ETHNICITY _OTHER']:
    colname_new = colname.replace(' [', '_').replace(']','').replace(' ','')
    print(colname_new)
    df.rename(columns = {colname: colname_new}, inplace=True)

RACE_OR_ETHNICITY_AMALA
RACE_OR_ETHNICITY_ASIAN
RACE_OR_ETHNICITY_BLACK
RACE_OR_ETHNICITY_HAWPI
RACE_OR_ETHNICITY_WHITE
RACE_OR_ETHNICITY_OTHER


In [5]:
# df.to_csv(r'M:\Data\OnBoard\Data and Reports\BART\As CSV\BART_Final_Database_Mar18_SUBMITTED_with_station_xy_with_first_board_last_alight_fixColname NO POUND OR SINGLE QUOTE.csv', index=False)

In [6]:
bart_dict = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\onboard-surveys\make-uniform\production\Dictionary for Standard Database.csv')
bart_dict = bart_dict.loc[(bart_dict.operator == 'BART') & (bart_dict.Survey_year == 2015)]

bart_dict_cat = bart_dict.loc[bart_dict.Generic_Response != 'NONCATEGORICAL']

for i in bart_dict_cat.Survey_Variable.unique():
    print(i, ', ', bart_dict_cat.loc[bart_dict_cat.Survey_Variable == i]['Generic_Variable'].unique(), ', ', i in list(df))
    df_part = df[['ID', i]]
    dict_part = bart_dict_cat.loc[bart_dict_cat.Survey_Variable == i]

    if i in ['OR_TRANSPORT_CODE', 'TRIP_START_TIME2_CODE', 'DE_PLACE_TYPE_CODE_FINAL',
               'DE_TRANSPORT_CODE', 'ENGLISH_FLUENCY_CODE', 'USED_CLIPPER_CODE', 'BART_TICKET_CODE',
               'HOW_PAID_FARE_CODE', 'RESP_GENDER_CODE', 'HH_INCOME_CODE',
               'OR_PLACE_TYPE_CODE_FINAL', 'PPL_IN_HH_CODE', 'TRIP_END_TIME2_CODE',
               'EMPLOYED_IN_HH_CODE']:
        dict_part.Survey_Response = dict_part.Survey_Response.apply(lambda x: int(x))

    join = df_part.merge(dict_part, left_on=i, right_on='Survey_Response', how='left')
    missing = join.loc[join.Survey_Variable.isnull()]
    
    if missing.shape[0] > 0:
        print(missing[i].unique())

OR_TRANSPORT_CODE ,  ['access_mode'] ,  True
[ 99 103]
WILL_GO2SCHOOL_TODAY ,  ['at_school_after_dest_purp'] ,  True
[nan 'Missing - Interviewer' 'Missing - Question Not Asked'
 'Missing - Dummy Record']
BEEN_2SCHOOL_TODAY ,  ['at_school_prior_to_orig_purp'] ,  True
[nan 'Missing - Interviewer' 'Missing - Question Not Asked'
 'Missing - Dummy Record']
WORK_AFTER_TRIP ,  ['at_work_after_dest_purp'] ,  True
[nan 'Missing - Interviewer' 'Missing - Question Not Asked'
 'Missing - Dummy Record']
WORK_BEFORE_TRIP ,  ['at_work_prior_to_orig_purp'] ,  True


C:\Users\ywang\Anaconda3\lib\site-packages\pandas\core\generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


[nan 'Missing - Interviewer' 'Missing - Question Not Asked'
 'Missing - Dummy Record']
TRIP_START_TIME2_CODE ,  ['depart_hour'] ,  True
[1000  888  104  101  102  105  103]
DE_PLACE_TYPE_CODE_FINAL ,  ['dest_purp'] ,  True
[ 98  99 103]
DE_TRANSPORT_CODE ,  ['egress_mode'] ,  True
[ 99 103]
ENGLISH_FLUENCY_CODE ,  ['eng_proficient'] ,  True
[ nan 102.   9. 103.]
USED_CLIPPER_CODE ,  ['fare_category'] ,  True
[ nan 102. 103.]
BART_TICKET_CODE ,  ['fare_category'] ,  True
[ nan   9.   8. 102. 103.]
HOW_PAID_FARE_CODE ,  ['fare_medium'] ,  True
[  9   8   7 102 103]
RESP_GENDER_CODE ,  ['gender'] ,  True
[102 103]
HISP_LATINO_SPANISH_CODE ,  ['hispanic'] ,  True
['104' '102' '105' '103']
HH_INCOME_CODE ,  ['household_income'] ,  True
[ 99 102 101 103]
SRVY_IN_ENGL_SPANISH ,  ['interview_language'] ,  True
['Unknown' 'Missing - Dummy Record']
LANG_OTHER_THAN_ENG ,  ['language_at_home_binary'] ,  True
['Missing - Respondent Refusal' 'Missing - Interviewer'
 'Missing - Dummy Record']
OR_PLAC

In [8]:
df.loc[df.VEH_IN_HH.str.contains('Missing')]

,ID,DATE_COMPLETED,TIME_COMPLETED,STANDARD_INTERV_INIT,STATION_SURVEYED_CODE,STATION_SURVEYED,Q1_START,OLD_ENOUGH4SURVEY,INTERCEPT_LOCATION_CODE,FIRST_ENTERED_BART_CODE,...,HOME_COUNTY_GIS,FARECATEGORYSUMMARY,FIRST_ENTER_BART_LAT,FIRST_ENTER_BART_LON,BART_EXIT_LAT,BART_EXIT_LON,FIRST_BOARD_LAT,FIRST_BOARD_LON,LAST_ALIGHT_LAT,LAST_ALIGHT_LON
19779,32925.0,4/1/2015,6:42:43 AM,HNN,STA8,Castro Valley,4/1/2015 6:32,YES,Intercepted at Entry Station,STA8,...,ALAMEDA,Missing - interviewer,37.691766,-122.074907,37.789394,-122.401088,37.691766,-122.074907,37.789394,-122.401088
19807,32970.0,4/1/2015,7:05:12 AM,HNN,STA8,Castro Valley,4/1/2015 6:59,YES,Intercepted at Entry Station,STA8,...,ALAMEDA,Missing - interviewer,37.691766,-122.074907,37.785101,-122.406742,37.691766,-122.074907,37.785101,-122.406742
19815,32983.0,4/1/2015,7:11:11 AM,HNN,STA8,Castro Valley,4/1/2015 7:06,YES,Intercepted at Entry Station,STA8,...,ALAMEDA,Missing - interviewer,37.691766,-122.074907,37.789394,-122.401088,37.691766,-122.074907,37.789394,-122.401088
19823,32999.0,4/1/2015,7:17:33 AM,HNN,STA8,Castro Valley,4/1/2015 7:12,YES,Intercepted at Entry Station,STA8,...,ALAMEDA,Missing - interviewer,37.691766,-122.074907,37.699711,-121.928224,37.691766,-122.074907,37.699711,-121.928224
19834,33016.0,4/1/2015,7:23:45 AM,HNN,STA8,Castro Valley,4/1/2015 7:18,YES,Intercepted at Entry Station,STA8,...,ALAMEDA,Missing - interviewer,37.691766,-122.074907,37.779532,-122.413788,37.691766,-122.074907,37.779532,-122.413788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46940,93022.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
46941,93023.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
46942,93024.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
46943,93025.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record


In [9]:
df.loc[(df.PPL_IN_HH_CODE == 104) | (df.PPL_IN_HH_CODE == 102) | (df.PPL_IN_HH_CODE == 103)]

,ID,DATE_COMPLETED,TIME_COMPLETED,STANDARD_INTERV_INIT,STATION_SURVEYED_CODE,STATION_SURVEYED,Q1_START,OLD_ENOUGH4SURVEY,INTERCEPT_LOCATION_CODE,FIRST_ENTERED_BART_CODE,...,HOME_COUNTY_GIS,FARECATEGORYSUMMARY,FIRST_ENTER_BART_LAT,FIRST_ENTER_BART_LON,BART_EXIT_LAT,BART_EXIT_LON,FIRST_BOARD_LAT,FIRST_BOARD_LON,LAST_ALIGHT_LAT,LAST_ALIGHT_LON
450,1008.0,2/18/2015,5:37:27 PM,GLC,STA27,Montgomery St.,2/18/2015 17:24,YES,Intercepted at Entry Station,STA27,...,SAN FRANCISCO,Regular,37.789394,-122.401088,37.765023,-122.419743,37.789394,-122.401088,37.765023,-122.419743
644,1319.0,2/19/2015,2:25:43 PM,GLC,STA33,Powell St.,2/19/2015 14:19,YES,Intercepted at Entry Station,STA33,...,SAN FRANCISCO,Regular,37.785101,-122.406742,37.752463,-122.418157,37.785101,-122.406742,37.752463,-122.418157
660,1343.0,2/19/2015,2:51:32 PM,GLC,STA33,Powell St.,2/19/2015 14:44,YES,Intercepted at Entry Station,STA33,...,SAN FRANCISCO,Regular,37.785101,-122.406742,37.789394,-122.401088,37.785101,-122.406742,37.789394,-122.401088
826,1619.0,2/20/2015,10:18:40 AM,HKO,STA44,West Oakland,2/20/2015 10:07,YES,TRANSFERRED_HERE,STA20,...,ALAMEDA,HVD,37.775128,-122.224225,37.779532,-122.413788,37.775128,-122.224225,37.779532,-122.413788
1019,1908.0,2/20/2015,2:24:26 PM,MAW,STA21,Glen Park,2/20/2015 14:14,YES,Intercepted at Entry Station,STA21,...,CONTRA COSTA,Regular,37.733046,-122.433776,38.017489,-121.945388,37.745975,-122.452438,38.017489,-121.945388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46940,93022.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
46941,93023.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
46942,93024.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
46943,93025.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record


In [10]:
df.loc[(df.EMPLOYED_IN_HH_CODE == 104) | (df.EMPLOYED_IN_HH_CODE == 102) | (df.EMPLOYED_IN_HH_CODE == 103)]

,ID,DATE_COMPLETED,TIME_COMPLETED,STANDARD_INTERV_INIT,STATION_SURVEYED_CODE,STATION_SURVEYED,Q1_START,OLD_ENOUGH4SURVEY,INTERCEPT_LOCATION_CODE,FIRST_ENTERED_BART_CODE,...,HOME_COUNTY_GIS,FARECATEGORYSUMMARY,FIRST_ENTER_BART_LAT,FIRST_ENTER_BART_LON,BART_EXIT_LAT,BART_EXIT_LON,FIRST_BOARD_LAT,FIRST_BOARD_LON,LAST_ALIGHT_LAT,LAST_ALIGHT_LON
89,320.0,2/17/2015,10:42:30 AM,DAT,STA33,Powell St.,2/17/2015 10:37,YES,Intercepted at Entry Station,STA33,...,SAN FRANCISCO,Unknown,37.785101,-122.406742,37.712798,-122.213186,37.785101,-122.406742,37.712798,-122.213186
238,600.0,2/17/2015,5:45:45 PM,LKL,STA27,Montgomery St.,2/17/2015 17:39,YES,Intercepted at Entry Station,STA27,...,SAN MATEO,Unknown,37.789394,-122.401088,37.600002,-122.386564,37.789394,-122.401088,37.600002,-122.386564
450,1008.0,2/18/2015,5:37:27 PM,GLC,STA27,Montgomery St.,2/18/2015 17:24,YES,Intercepted at Entry Station,STA27,...,SAN FRANCISCO,Regular,37.789394,-122.401088,37.765023,-122.419743,37.789394,-122.401088,37.765023,-122.419743
726,1450.0,2/19/2015,6:47:03 PM,MMB,STA27,Montgomery St.,2/19/2015 18:35,YES,Intercepted at Entry Station,STA27,...,ALAMEDA,Unknown,37.789394,-122.401088,37.696991,-122.126455,37.789394,-122.401088,37.696991,-122.126455
727,1452.0,2/19/2015,6:47:03 PM,MMB,STA27,Montgomery St.,2/19/2015 18:35,YES,Intercepted at Entry Station,STA27,...,ALAMEDA,Unknown,37.789394,-122.401088,37.696991,-122.126455,37.789394,-122.401088,37.696991,-122.126455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46941,93023.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
46942,93024.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
46943,93025.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
46944,93026.3,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,103,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,STA3,...,Missing - Dummy,Missing - Dummy record,37.808119,-122.268879,37.793575,-122.395795,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record,Missing - Dummy Record
